# Encontro 10: Lacunas Estruturais

O enunciado da *Escrita 4* continua ao longo deste notebook.

**Preste atenção nas partes em negrito.**

Importando a biblioteca:

In [17]:
import sys
sys.path.append('..')

import socnet as sn

Configurando a biblioteca:

In [18]:
sn.node_size = 10
sn.node_color = (0, 0, 0)

sn.edge_width = 1

Gerando um grafo completo:

In [19]:
g = sn.generate_complete_graph(15)

sn.show_graph(g)

**Esse será o grafo da comunidade.**

Atribuindo aleatoriamente tipos aos nós:

In [20]:
from random import shuffle


def randomize_types(g, num_openers, num_closers, num_chummies):
    if num_openers + num_closers + num_chummies != g.number_of_nodes():
        raise Exception('a soma dos tipos não é igual ao número de nós')

    nodes = g.nodes()

    shuffle(nodes)

    for _ in range(num_openers):
        g.node[nodes.pop()]['type'] = 'opener'

    for _ in range(num_closers):
        g.node[nodes.pop()]['type'] = 'closer'

    for _ in range(num_chummies):
        g.node[nodes.pop()]['type'] = 'chummy'


randomize_types(g, 15, 0, 0)

Atribuindo aleatoriamente existências às arestas:

In [21]:
from random import random


def randomize_existences(g):
    for n, m in g.edges():
        if random() < 0.5:
            g.edge[n][m]['exists'] = 0
        else:
            g.edge[n][m]['exists'] = 1


randomize_existences(g)

Convertendo tipos e existências em cores para visualização:

In [22]:
def convert_types_and_existences_to_colors(g):
    for n in g.nodes():
        if g.node[n]['type'] == 'opener':
            g.node[n]['color'] = (255, 0, 0)
        elif g.node[n]['type'] == 'closer':
            g.node[n]['color'] = (0, 255, 0)
        else:
            g.node[n]['color'] = (0, 0, 255)

    for n, m in g.edges():
        if g.edge[n][m]['exists'] == 0:
            g.edge[n][m]['color'] = (192, 192, 192)
        else:
            g.edge[n][m]['color'] = (0, 0, 0)


convert_types_and_existences_to_colors(g)

sn.show_graph(g)

Definindo uma medida de restrição:

In [23]:
def neighbors(g, n):
    return [m for m in g.neighbors(n) if g.edge[n][m]['exists'] == 1]


# Independentemente do tipo, a restrição é sempre entre 0 e 2.

def calculate_constraint(g, n):
    neighbors_n = neighbors(g, n)
    degree_n = len(neighbors_n)

    # Todos os tipos evitam isolamento. A restrição é máxima nesse caso.

    if degree_n == 0:
        return 2

    # Para um chummy, a restrição é o inverso do grau. Uma pequena
    # normalização é necessária para garantir que está entre 0 e 2.

    if g.node[n]['type'] == 'chummy':
        return 2 * (g.number_of_nodes() - degree_n - 1) / (g.number_of_nodes() - 1)

    # Fórmula de Burt.

    constraint = 0

    for m in neighbors_n:
        neighbors_m = neighbors(g, m)
        degree_m = len(neighbors_m)

        sub_constraint = 1 / degree_n

        for l in neighbors_m:
            if n != l and g.edge[n][l]['exists'] == 1:
                sub_constraint += (1 / degree_n) * (1 / degree_m)

        constraint += sub_constraint ** 2

    # Para um closer, a restrição é o inverso da fórmula de Burt.

    if g.node[n]['type'] == 'closer':
        return 2 - constraint

    # Para um opener, a restrição é a fórmula de Burt.

    return constraint

Definindo uma atualização de existência:

In [24]:
from random import choice


def equals(a, b):
    return abs(a - b) < 0.000000001


def invert_existence(g, n, m):
    g.edge[n][m]['exists'] = 1 - g.edge[n][m]['exists']


def update_existences(g):

    # Para cada nó n...

    for n in g.nodes():

        # Calcula a restrição de n.

        cn = calculate_constraint(g, n)

        # Inicializa o dicionário de ganhos.

        g.node[n]['gains'] = {}

        # Para cada nó m diferente de n...

        for m in g.nodes():
            if n != m:

                # Calcula a restrição de m.

                cm = calculate_constraint(g, m)

                # Inverte temporariamente a existência de (n, m) para ver o que acontece.

                invert_existence(g, n, m)

                # Se a inversão representa uma adição e ela não faz a restrição
                # de m diminuir, então o ganho é zero porque essa inversão não
                # é possível: adicionar só é possível se ambos os nós querem.

                if g.edge[n][m]['exists'] == 1 and calculate_constraint(g, m) >= cm:
                    g.node[n]['gains'][m] = 0

                # Senão, o ganho é simplesmente a diferença das restrições.

                else:
                    g.node[n]['gains'][m] = cn - calculate_constraint(g, n)

                # Restaura a existência original de (n, m), pois a inversão era temporária.

                invert_existence(g, n, m)

        # Obtém o maior ganho de n.

        g.node[n]['max_gain'] = max(g.node[n]['gains'].values())

    # Obtém o maior ganho de todos os nós.

    max_gain = max([g.node[n]['max_gain'] for n in g.nodes()])

    # Se o maior ganho não for positivo, devolve False indicando que o grafo estabilizou.

    if max_gain <= 0:
        return False

    # Senão, escolhe aleatoriamente uma aresta correspondente ao maior ganho e inverte sua existência.

    n = choice([n for n in g.nodes() if equals(g.node[n]['max_gain'], max_gain)])

    m = choice([m for m in g.node[n]['gains'] if equals(g.node[n]['gains'][m], max_gain)])

    invert_existence(g, n, m)

    # Devolve True indicando que o grafo ainda não estabilizou.

    return True

Definindo uma calculadora de variáveis.

In [25]:
from math import inf
from statistics import stdev
from queue import Queue


def calculate_variables(g, verbose=False):

    # Cria uma cóṕia do grafo na qual as arestas realmente
    # existem ou não existem. Isso facilita os cálculos.

    gc = g.copy()

    for n, m in g.edges():
        if g.edge[n][m]['exists'] == 0:
            gc.remove_edge(n, m)

    # Cálculo do número de arestas. (densidade)

    num_edges = gc.number_of_edges()

    if verbose:
        print('número de arestas:', num_edges)

    # Cálculo do número de componentes. (fragmentação)

    for n in gc.nodes():
        gc.node[n]['label'] = 0

    label = 0
    q = Queue()

    for s in gc.nodes():
        if gc.node[s]['label'] == 0:
            label += 1

            gc.node[s]['label'] = label
            q.put(s)

            while not q.empty():
                n = q.get()

                for m in gc.neighbors(n):
                    if gc.node[m]['label'] == 0:
                        gc.node[m]['label'] = label
                        q.put(m)

    num_components = label

    if verbose:
        print('número de componentes:', num_components)

    # Cálculo do desvio do tamanho de componentes. (fragmentação)

    sizes = {label: 0 for label in range(1, num_components + 1)}

    for n in gc.nodes():
        sizes[gc.node[n]['label']] += 1

    if num_components == 1:
        dev_components = 0
    else:
        dev_components = stdev(sizes.values())

    if verbose:
        print('desvio do tamanho de componentes: {:05.2f}\n'.format(dev_components))

    # Cálculo do desvio do betweenness. (desigualdade)
    # Cálculo do betweenness médio por tipo. (quais perfis ficaram centrais)

    sn.build_betweenness(gc)

    betweenness = []

    mean_betweenness = {
        'closer': 0,
        'opener': 0,
        'chummy': 0,
    }

    for n in gc.nodes():
        betweenness.append(gc.node[n]['theoretical_betweenness'])

        mean_betweenness[gc.node[n]['type']] += betweenness[-1]

        if verbose:
            print('betweenness do nó {:2} ({}): {:05.2f}'.format(n, gc.node[n]['type'], betweenness[-1]))

    dev_betweenness = stdev(betweenness)

    for key in mean_betweenness:
        length = len([n for n in gc.nodes() if gc.node[n]['type'] == key])

        if length == 0:
            mean_betweenness[key] = 0
        else:
            mean_betweenness[key] /= length

    if verbose:
        print('\ndesvio do betweenness: {:05.2f}\n'.format(dev_betweenness))

        for key, value in mean_betweenness.items():
            print('betweenness médios de {}: {:05.2f}'.format(key, value))

    return num_edges, num_components, dev_components, dev_betweenness, mean_betweenness

Simulando várias vezes o processo no qual os nós invertem a existência de arestas até não poderem mais diminuir a restrição.

In [26]:
TIMES = 25

No final das simulações, a média das variáveis é impressa.

In [27]:
num_openers  = 15
num_closers  =  0
num_chummies =  0

mean_num_edges = 0
mean_num_components = 0
mean_dev_components = 0
mean_dev_betweenness = 0

mean_mean_betweenness = {
    'opener': 0,
    'closer': 0,
    'chummy': 0,
}


for _ in range(TIMES):
    randomize_types(g, num_openers, num_closers, num_chummies)

    randomize_existences(g)

    while update_existences(g):
        pass

    num_edges, num_components, dev_components, dev_betweenness, mean_betweenness = calculate_variables(g)

    mean_num_edges += num_edges
    mean_num_components += num_components
    mean_dev_components += dev_components
    mean_dev_betweenness += dev_betweenness

    for key, value in mean_betweenness.items():
        mean_mean_betweenness[key] += value

mean_num_edges /= TIMES
mean_num_components /= TIMES
mean_dev_components /= TIMES
mean_dev_betweenness /= TIMES

for key in mean_mean_betweenness:
    mean_mean_betweenness[key] /= TIMES

print('média do número de arestas: {:05.2f}'.format(mean_num_edges))
print('média do número de componentes: {:05.2f}'.format(mean_num_components))
print('média do desvio do tamanho de componentes: {:05.2f}'.format(mean_dev_components))
print('média do desvio do betweenness: {:05.2f}'.format(mean_dev_betweenness))

for key, value in mean_mean_betweenness.items():
    print('média do betweenness médio de {}: {:05.2f}'.format(key, value))

média do número de arestas: 55.04
média do número de componentes: 01.00
média do desvio do tamanho de componentes: 00.00
média do desvio do betweenness: 00.83
média do betweenness médio de chummy: 00.00
média do betweenness médio de opener: 03.33
média do betweenness médio de closer: 00.00


Para ter *insights* sobre o que está acontecendo, não esqueça de examinar a versão animada da simulação!

In [28]:
def update_positions(g, invert=False):
    if invert:
        for n, m in g.edges():
            g.edge[n][m]['notexists'] = 1 - g.edge[n][m]['exists']

        sn.update_positions(g, 'notexists')
    else:
        sn.update_positions(g, 'exists')


def snapshot(g, frames):
    convert_types_and_existences_to_colors(g)

    frame = sn.generate_frame(g)

    frames.append(frame)


frames = []

randomize_types(g, num_openers, num_closers, num_chummies)

randomize_existences(g)

sn.randomize_positions(g)

snapshot(g, frames)

while update_existences(g):
    update_positions(g, True)

    snapshot(g, frames)

sn.show_animation(frames)

_, _, _, _, _ = calculate_variables(g, verbose=True)

número de arestas: 46
número de componentes: 1
desvio do tamanho de componentes: 00.00

betweenness do nó  0 (opener): 03.72
betweenness do nó  1 (opener): 03.02
betweenness do nó  2 (opener): 03.67
betweenness do nó  3 (opener): 05.20
betweenness do nó  4 (opener): 03.02
betweenness do nó  5 (opener): 03.02
betweenness do nó  6 (opener): 04.50
betweenness do nó  7 (opener): 03.72
betweenness do nó  8 (opener): 04.50
betweenness do nó  9 (opener): 03.67
betweenness do nó 10 (opener): 03.02
betweenness do nó 11 (opener): 04.50
betweenness do nó 12 (opener): 03.72
betweenness do nó 13 (opener): 05.20
betweenness do nó 14 (opener): 04.50

desvio do betweenness: 00.76

betweenness médios de chummy: 00.00
betweenness médios de opener: 03.93
betweenness médios de closer: 00.00


Se a animação não parece dizer muito, tente trocar `False` por `True` no `update_positions`.

In [39]:
def calc(mdic, num_openers, num_closers, num_chummies):
    mean_num_edges = 0
    mean_num_components = 0
    mean_dev_components = 0
    mean_dev_betweenness = 0

    mean_mean_betweenness = {
        'opener': 0,
        'closer': 0,
        'chummy': 0,
    }

    TIMES = 25

    for _ in range(TIMES):
        randomize_types(g, num_openers, num_closers, num_chummies)

        randomize_existences(g)

        while update_existences(g):
            pass

        num_edges, num_components, dev_components, dev_betweenness, mean_betweenness = calculate_variables(g)

        mean_num_edges += num_edges
        mean_num_components += num_components
        mean_dev_components += dev_components
        mean_dev_betweenness += dev_betweenness

        for key, value in mean_betweenness.items():
            mean_mean_betweenness[key] += value

    mean_num_edges /= TIMES
    mean_num_components /= TIMES
    mean_dev_components /= TIMES
    mean_dev_betweenness /= TIMES

    for key in mean_mean_betweenness:
        mean_mean_betweenness[key] /= TIMES

    print('média do número de arestas: {:05.2f}'.format(mean_num_edges))
    print('média do número de componentes: {:05.2f}'.format(mean_num_components))
    print('média do desvio do tamanho de componentes: {:05.2f}'.format(mean_dev_components))
    print('média do desvio do betweenness: {:05.2f}'.format(mean_dev_betweenness))

    for key, value in mean_mean_betweenness.items():
        print('média do betweenness médio de {}: {:05.2f}'.format(key, value))
        
    mdic['mean_num_edges'].append(mean_num_edges)
    mdic['mean_num_components'].append(mean_num_components)
    mdic['mean_dev_components'].append(mean_dev_components)
    mdic['mean_dev_betweenness'].append(mean_dev_betweenness)
    mdic['mean_mean_betweenness_chummy'].append(mean_mean_betweenness['chummy'])
    mdic['mean_mean_betweenness_opener'].append(mean_mean_betweenness['opener'])
    mdic['mean_mean_betweenness_closer'].append(mean_mean_betweenness['closer'])

In [46]:
# openers zerados
fon = {}
mdic = {
    'mean_num_edges': [],
    'mean_num_components': [],
    'mean_dev_components': [],
    'mean_dev_betweenness': [],
    'mean_mean_betweenness_chummy': [],
    'mean_mean_betweenness_opener': [],
    'mean_mean_betweenness_closer': []
}

num_openers  =  0
num_closers  = 15
num_chummies =  0

print('Openers Zerados:')
for i in range(16):
    calc(mdic, num_openers, num_closers - i, num_chummies + i)
    
for key, value in mdic.items():
    print('{}: {}'.format(key, sum(value) / 15))
    
fon['opener'] = mdic

Openers Zerados:
média do número de arestas: 15.80
média do número de componentes: 05.88
média do desvio do tamanho de componentes: 01.26
média do desvio do betweenness: 00.00
média do betweenness médio de chummy: 00.00
média do betweenness médio de opener: 00.00
média do betweenness médio de closer: 00.00
média do número de arestas: 18.36
média do número de componentes: 02.28
média do desvio do tamanho de componentes: 03.54
média do desvio do betweenness: 08.56
média do betweenness médio de chummy: 33.16
média do betweenness médio de opener: 00.00
média do betweenness médio de closer: 00.00
média do número de arestas: 17.52
média do número de componentes: 02.52
média do desvio do tamanho de componentes: 03.41
média do desvio do betweenness: 07.75
média do betweenness médio de chummy: 21.38
média do betweenness médio de opener: 00.00
média do betweenness médio de closer: 00.00
média do número de arestas: 16.56
média do número de componentes: 02.40
média do desvio do tamanho de componen

In [47]:
# closer zerados

num_openers  = 15
num_closers  =  0
num_chummies =  0

zdic = {
    'mean_num_edges': [],
    'mean_num_components': [],
    'mean_dev_components': [],
    'mean_dev_betweenness': [],
    'mean_mean_betweenness_chummy': [],
    'mean_mean_betweenness_opener': [],
    'mean_mean_betweenness_closer': []
}

for i in range(16):
    calc(zdic, num_openers - i, num_closers, num_chummies + i)
    
    
for key, value in zdic.items():
    print('{}: {}'.format(key, sum(value) / 15))
    
fon['closer'] = zdic

média do número de arestas: 55.20
média do número de componentes: 01.00
média do desvio do tamanho de componentes: 00.00
média do desvio do betweenness: 01.01
média do betweenness médio de chummy: 00.00
média do betweenness médio de opener: 03.32
média do betweenness médio de closer: 00.00
média do número de arestas: 55.68
média do número de componentes: 01.00
média do desvio do tamanho de componentes: 00.00
média do desvio do betweenness: 00.91
média do betweenness médio de chummy: 04.02
média do betweenness médio de opener: 03.24
média do betweenness médio de closer: 00.00
média do número de arestas: 56.08
média do número de componentes: 01.00
média do desvio do tamanho de componentes: 00.00
média do desvio do betweenness: 01.05
média do betweenness médio de chummy: 03.39
média do betweenness médio de opener: 03.24
média do betweenness médio de closer: 00.00
média do número de arestas: 56.76
média do número de componentes: 01.00
média do desvio do tamanho de componentes: 00.00
média 

In [48]:
# chummy zerados

num_openers  =  0
num_closers  = 15
num_chummies =  0

xdic = {
    'mean_num_edges': [],
    'mean_num_components': [],
    'mean_dev_components': [],
    'mean_dev_betweenness': [],
    'mean_mean_betweenness_chummy': [],
    'mean_mean_betweenness_opener': [],
    'mean_mean_betweenness_closer': []
}

for i in range(16):
    calc(xdic, num_openers + i, num_closers - i, num_chummies)
    
    
for key, value in xdic.items():
    print('{}: {}'.format(key, sum(value) / 15))
    
fon['chummy'] = xdic

média do número de arestas: 16.00
média do número de componentes: 05.84
média do desvio do tamanho de componentes: 01.29
média do desvio do betweenness: 00.00
média do betweenness médio de chummy: 00.00
média do betweenness médio de opener: 00.00
média do betweenness médio de closer: 00.00
média do número de arestas: 17.92
média do número de componentes: 02.40
média do desvio do tamanho de componentes: 03.38
média do desvio do betweenness: 07.99
média do betweenness médio de chummy: 00.00
média do betweenness médio de opener: 30.96
média do betweenness médio de closer: 00.00
média do número de arestas: 17.04
média do número de componentes: 02.60
média do desvio do tamanho de componentes: 03.37
média do desvio do betweenness: 08.26
média do betweenness médio de chummy: 00.00
média do betweenness médio de opener: 22.06
média do betweenness médio de closer: 00.00
média do número de arestas: 15.76
média do número de componentes: 02.40
média do desvio do tamanho de componentes: 04.24
média 

In [49]:
print(fon)

{'chummy': {'mean_mean_betweenness_opener': [0.0, 30.96, 22.06, 20.786666666666665, 23.22, 21.944000000000003, 19.12666666666667, 16.662857142857145, 15.615, 12.680000000000001, 10.456000000000001, 8.647272727272725, 7.203333333333335, 5.781538461538461, 4.511428571428572, 3.3279999999999994], 'mean_dev_components': [1.2867685042275125, 3.3786092765434983, 3.374992247156982, 4.239724864041556, 4.3421440217854, 3.697932500870245, 2.9179302687454314, 2.187745084230598, 0.8768124086713189, 0.8202438661763952, 0.9333809511662428, 0.3111269837220809, 0.0, 0.0, 0.0, 0.0], 'mean_dev_betweenness': [0.0, 7.993837626572107, 8.255881465689377, 9.326837469945474, 11.674315310377072, 11.840349223055533, 11.340058935007715, 10.490462381802326, 10.316257558649209, 9.086044112787947, 8.330429324689952, 7.603943167925264, 6.309368624469203, 5.034177082479003, 3.61137009166971, 0.9182049668035098], 'mean_num_components': [5.84, 2.4, 2.6, 2.4, 2.0, 1.68, 1.52, 1.4, 1.12, 1.12, 1.12, 1.04, 1.0, 1.0, 1.0, 

In [50]:
for key, value in fon.items():
    print('{}[mean_num_edges]: {}'.format(key, value['mean_num_edges']))
    
    

chummy[mean_num_edges]: [16.0, 17.92, 17.04, 15.76, 15.8, 16.64, 18.0, 20.08, 22.8, 25.84, 29.24, 33.72, 38.24, 42.64, 48.12, 55.08]
opener[mean_num_edges]: [15.8, 18.36, 17.52, 16.56, 17.96, 20.36, 24.44, 28.76, 35.0, 41.96, 49.8, 59.0, 68.96, 80.0, 92.52, 105.0]
closer[mean_num_edges]: [55.2, 55.68, 56.08, 56.76, 59.28, 61.2, 64.76, 66.56, 70.36, 77.48, 86.8, 94.24, 99.12, 102.64, 104.0, 105.0]


In [51]:
for key, value in fon.items():
    print('{}[mean_mean_betweenness_chummy]: {}'.format(key, value['mean_mean_betweenness_chummy']))

chummy[mean_mean_betweenness_chummy]: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
opener[mean_mean_betweenness_chummy]: [0.0, 33.16, 21.38, 17.85333333333334, 12.85, 10.703999999999999, 8.686666666666666, 8.308571428571426, 8.265, 6.799999999999999, 5.816000000000002, 4.5454545454545485, 3.1533333333333333, 2.0, 0.8914285714285711, 0.0]
closer[mean_mean_betweenness_chummy]: [0.0, 4.019142857142857, 3.3866666666666667, 3.4094179894179892, 3.1206349206349207, 3.0559333333333343, 2.849238095238095, 2.6294013605442172, 2.2819365079365084, 1.8251282667949338, 1.1712914862914863, 0.7430928162746345, 0.42006837606837616, 0.16970414201183426, 0.07142857142857141, 0.0]


In [1]:
import matplotlib.pyplot as plt

In [4]:
import ast

fon = {'chummy': {'mean_mean_betweenness_opener': [0.0, 30.96, 22.06, 20.786666666666665, 23.22, 21.944000000000003, 19.12666666666667, 16.662857142857145, 15.615, 12.680000000000001, 10.456000000000001, 8.647272727272725, 7.203333333333335, 5.781538461538461, 4.511428571428572, 3.3279999999999994], 'mean_dev_components': [1.2867685042275125, 3.3786092765434983, 3.374992247156982, 4.239724864041556, 4.3421440217854, 3.697932500870245, 2.9179302687454314, 2.187745084230598, 0.8768124086713189, 0.8202438661763952, 0.9333809511662428, 0.3111269837220809, 0.0, 0.0, 0.0, 0.0], 'mean_dev_betweenness': [0.0, 7.993837626572107, 8.255881465689377, 9.326837469945474, 11.674315310377072, 11.840349223055533, 11.340058935007715, 10.490462381802326, 10.316257558649209, 9.086044112787947, 8.330429324689952, 7.603943167925264, 6.309368624469203, 5.034177082479003, 3.61137009166971, 0.9182049668035098], 'mean_num_components': [5.84, 2.4, 2.6, 2.4, 2.0, 1.68, 1.52, 1.4, 1.12, 1.12, 1.12, 1.04, 1.0, 1.0, 1.0, 1.0], 'mean_num_edges': [16.0, 17.92, 17.04, 15.76, 15.8, 16.64, 18.0, 20.08, 22.8, 25.84, 29.24, 33.72, 38.24, 42.64, 48.12, 55.08], 'mean_mean_betweenness_closer': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'mean_mean_betweenness_chummy': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, 'opener': {'mean_mean_betweenness_opener': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'mean_dev_components': [1.2566627284734193, 3.5415291082237617, 3.4123346821083773, 4.431263915717688, 3.8311717721000558, 4.557183957539829, 4.533710032568221, 4.144738916180638, 3.1112698372208087, 2.4041630560342617, 1.5556349186104046, 0.2545584412271571, 0.3111269837220809, 0.0, 0.0, 0.0], 'mean_dev_betweenness': [0.0, 8.56187518401586, 7.74891301610932, 8.086948901839655, 6.762738525212747, 6.835935055390845, 6.02692942806565, 6.93142068909263, 7.093885515196589, 6.780728760078484, 6.627038421963774, 6.303945073941991, 5.347902370672541, 4.574255078568481, 3.22232214404457, 0.0], 'mean_num_components': [5.88, 2.28, 2.52, 2.4, 2.48, 2.32, 2.2, 1.96, 1.48, 1.36, 1.2, 1.04, 1.04, 1.0, 1.0, 1.0], 'mean_num_edges': [15.8, 18.36, 17.52, 16.56, 17.96, 20.36, 24.44, 28.76, 35.0, 41.96, 49.8, 59.0, 68.96, 80.0, 92.52, 105.0], 'mean_mean_betweenness_closer': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'mean_mean_betweenness_chummy': [0.0, 33.16, 21.38, 17.85333333333334, 12.85, 10.703999999999999, 8.686666666666666, 8.308571428571426, 8.265, 6.799999999999999, 5.816000000000002, 4.5454545454545485, 3.1533333333333333, 2.0, 0.8914285714285711, 0.0]}, 'closer': {'mean_mean_betweenness_opener': [3.3199999999999994, 3.235775510204081, 3.2420512820512823, 3.167645502645503, 3.021587301587301, 2.852033333333334, 2.5716190476190475, 2.5042738095238084, 2.340643990929705, 1.8489742664742659, 1.2974170274170274, 0.6464947552447552, 0.2797264957264957, 0.07692307692307693, 0.0, 0.0], 'mean_dev_components': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'mean_dev_betweenness': [1.009496695134969, 0.9070454966439443, 1.0530360347030439, 1.3255295617957532, 0.8025691437735141, 0.8943373743876205, 0.6022707203642444, 0.5211400782424059, 0.2965147627771358, 0.3412008829833681, 0.3371798018789015, 0.10958661789697473, 0.07110701465652274, 0.05087889305629189, 0.02706659809803835, 0.0], 'mean_num_components': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'mean_num_edges': [55.2, 55.68, 56.08, 56.76, 59.28, 61.2, 64.76, 66.56, 70.36, 77.48, 86.8, 94.24, 99.12, 102.64, 104.0, 105.0], 'mean_mean_betweenness_closer': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'mean_mean_betweenness_chummy': [0.0, 4.019142857142857, 3.3866666666666667, 3.4094179894179892, 3.1206349206349207, 3.0559333333333343, 2.849238095238095, 2.6294013605442172, 2.2819365079365084, 1.8251282667949338, 1.1712914862914863, 0.7430928162746345, 0.42006837606837616, 0.16970414201183426, 0.07142857142857141, 0.0]}}


In [5]:
fon['closer']

{'mean_dev_betweenness': [1.009496695134969,
  0.9070454966439443,
  1.0530360347030439,
  1.3255295617957532,
  0.8025691437735141,
  0.8943373743876205,
  0.6022707203642444,
  0.5211400782424059,
  0.2965147627771358,
  0.3412008829833681,
  0.3371798018789015,
  0.10958661789697473,
  0.07110701465652274,
  0.05087889305629189,
  0.02706659809803835,
  0.0],
 'mean_dev_components': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'mean_mean_betweenness_chummy': [0.0,
  4.019142857142857,
  3.3866666666666667,
  3.4094179894179892,
  3.1206349206349207,
  3.0559333333333343,
  2.849238095238095,
  2.6294013605442172,
  2.2819365079365084,
  1.8251282667949338,
  1.1712914862914863,
  0.7430928162746345,
  0.42006837606837616,
  0.16970414201183426,
  0.07142857142857141,
  0.0],
 'mean_mean_betweenness_closer': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]

In [10]:
for key, value in fon.items():
    for k, v in value.items():
        fig, _ = plt.subplots()
        plt.ylabel('{}: {}'.format(key, k))
        plt.plot(v)
#         plt.show()
        plt.savefig('{}-{}.png'.format(key, k))
        plt.close(fig)